# Comparison of an SMPI trace with a real trace for HPL

In [1]:
import io
import zipfile
import pandas
from plotnine import *
import plotnine
plotnine.options.figure_size = (12, 8)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # removing annoying Pandas warning

def read_csv(archive_name, csv_name, columns=None, filter_func=lambda x: x):
    archive = zipfile.ZipFile(archive_name)
    df= pandas.read_csv(io.BytesIO(filter_func(archive.read(csv_name))), names=columns)
    df.columns = df.columns.str.strip()
    return df

def filter_pajeng(pajeng_output, keep):
    lines = pajeng_output.split(b'\n')
    lines = [l for l in lines if l.startswith(keep.encode())]
    return b'\n'.join(lines)

def _read_paje_state(archive_name, csv_name, has_msg_size=False, replace_func=('MPI', 'MPI')):
    columns=['type', 'rank', 'container', 'start', 'end', 'duration', 'level', 'function']
    if has_msg_size:
        columns.append('tmp')
    df = read_csv(archive_name, csv_name, columns=columns, filter_func=lambda x: filter_pajeng(x, 'State'))
    df['function'] = df['function'].str.replace(*replace_func).str.strip()  # for some reason Simgrid uses PMPI_Wait and not MPI_Wait
    df['msg_size'] = -1
    if has_msg_size:
        df.loc[df['function'].isin(['MPI_Recv', 'MPI_Send', 'MPI_Isend', 'MPI_Irecv']), 'msg_size'] = df['tmp']
        df['msg_size'] = df['msg_size'].astype(int)
    df['rank'] = df['rank'].str.slice(5).astype(int)  # changing 'rank42' into 42
    return df

def read_mpi_state(archive_name, csv_name):
    df = _read_paje_state(archive_name, csv_name, True)
    return df

def read_smpi_state(archive_name, csv_name):
    df = _read_paje_state(archive_name, csv_name, True, ('PMPI', 'MPI'))
    df['rank'] = -df['rank']  # in SMPI, we have 'rank-42' and not 'rank42', so there remains a - that we should remove
    return df

def read_smpi_link(archive_name, csv_name):
    df = read_csv(archive_name, csv_name,
                  columns=['type', 'level', 'container', 'start', 'end', 'duration', 'commType', 'src', 'dst', 'msg_size'],
                  filter_func=lambda x: filter_pajeng(x, 'Link'))
    df['src'] = df['src'].str.slice(6).astype(int)  # changing 'rank42' into 42
    df['dst'] = df['dst'].str.slice(6).astype(int)  # changing 'rank42' into 42
#    df['remote'] = (df['src'] // 32) != (df['dst'] // 32)
    return df

def read_archive(archive_name, mode, drop_func=['MPI_Comm_rank', 'MPI_Comm_size', 'MPI_Comm_split', 'MPI_Comm_free', 'MPI_Init', 'MPI_Finalize']):
    assert mode in ('reality', 'simgrid')
    print('# Mode: %s' % mode)
    blas_trace = read_csv(archive_name, 'trace_blas_0.csv')
    print('\tBLAS trace: %6d lines' % len(blas_trace))
    blas_trace['start'] = blas_trace.timestamp
    blas_trace['end'] = blas_trace.timestamp + blas_trace.duration
    blas_trace = blas_trace[['function', 'start', 'end', 'rank', 'm', 'n', 'k']]
    blas_trace['msg_size'] = -1
    blas_trace['kind'] = 'BLAS'
    if mode == 'reality':
        mpi_trace = read_mpi_state(archive_name, 'trace_mpi_0.csv')
    else:
        mpi_trace = read_smpi_state(archive_name, 'trace_mpi_0.csv')
    print('\tMPI  trace: %6d lines' % len(mpi_trace))
    mpi_trace = mpi_trace[['function', 'start', 'end', 'rank', 'msg_size']]
    mpi_trace['kind'] = 'MPI'
    df = pandas.concat([blas_trace, mpi_trace])
    df['mode'] = mode
    df['rank+1'] = df['rank'] + 1
    old_len = len(df)
    df = df[~df['function'].str.strip().isin(drop_func)]
    print('Removed %d events with functions in %s' % (old_len-len(df), ', '.join(drop_func)))
    if mode == 'reality':
        df_links = None
    else:
        df_links = read_smpi_link(archive_name, 'trace_mpi_0.csv')
    df['duration'] = df['end'] - df['start']
    return df, df_links

reality_file = 'grenoble_2018-12-13_1827231.zip'
simgrid_file = '../smpi_hpl/grenoble_2018-12-13_1827440.zip'

trace_reality, link_reality = read_archive(reality_file, 'reality')
trace_simgrid, link_simgrid = read_archive(simgrid_file, 'simgrid')
traces = pandas.concat([trace_reality, trace_simgrid])

traces.head()

# Mode: reality
	BLAS trace: 1996880 lines
	MPI  trace: 3591458 lines
Removed 1997408 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize
# Mode: simgrid
	BLAS trace: 2640826 lines
	MPI  trace: 4684421 lines
Removed 16 events with functions in MPI_Comm_rank, MPI_Comm_size, MPI_Comm_split, MPI_Comm_free, MPI_Init, MPI_Finalize


,end,function,k,kind,m,msg_size,n,rank,start,mode,rank+1,duration
0,173.488034,dtrsm,-1.0,BLAS,2.0,-1,2.0,0,173.488,reality,1,0.000034
1,173.488296,dgemm,2.0,BLAS,125054.0,-1,2.0,0,173.488,reality,1,0.000296
2,173.489000,dtrsm,-1.0,BLAS,0.0,-1,2.0,0,173.489,reality,1,0.000000
3,173.489000,dgemm,2.0,BLAS,125052.0,-1,0.0,0,173.489,reality,1,0.000000
4,173.489002,dtrsm,-1.0,BLAS,4.0,-1,4.0,0,173.489,reality,1,0.000002


## HPL result

In [2]:
res = read_csv(reality_file, 'results.csv')
print('%s Gflops' % res['gflops'].max())
print('%s seconds' % res['time'].max())
res

4291.0 Gflops
2427.83 seconds


,proc_p,swap,gflops,proc_q,bcast,time,rfact,depth,pfact,process_per_node,residual,matrix_size,mem_align,block_size,thread_per_process
0,2,0,4291.0,4,2,2427.83,2,1,1,1,0.001305,250000,8,128,32


In [3]:
res = read_csv(simgrid_file, 'results.csv')
print('%s Gflops' % res['gflops'].max())
print('%s seconds' % res['time'].max())
res

5201.0 Gflops
2002.97 seconds


,minor_page_fault,dgemm_intercept,simulation_time,dtrsm_coefficient,time,thread_per_process,proc_q,bcast,application_time,block_size,...,swap,cpu_utilization,matrix_size,dgemm_coefficient,dtrsm_intercept,process_per_node,usr_time,depth,sys_time,pfact
0,3337905,0.000099,129.215,4.524788e-12,2002.97,32,4,2,13.9289,128,...,0,0.99,250000,2.548998e-12,0.00004,1,104.99,1,24.16,1


## Checking the parameters

In [4]:
read_csv(reality_file, 'exp.csv')

,matrix_size,block_size,proc_p,proc_q,pfact,rfact,bcast,depth,swap,mem_align,process_per_node,thread_per_process
0,250000,128,2,4,1,2,2,1,0,8,1,32


In [5]:
read_csv(simgrid_file, 'exp_smpi.csv')

,matrix_size,block_size,proc_p,proc_q,pfact,rfact,bcast,depth,swap,mem_align,process_per_node,thread_per_process,dgemm_coefficient,dgemm_intercept,dtrsm_coefficient,dtrsm_intercept
0,250000,128,2,4,1,2,2,1,0,8,1,32,2.548998e-12,0.000099,4.524788e-12,0.00004


In [6]:
print(zipfile.ZipFile(simgrid_file).read('dahu.xml').decode('ascii'))

<?xml version="1.0"?>
<!DOCTYPE platform SYSTEM "http://simgrid.gforge.inria.fr/simgrid/simgrid.dtd">

<platform version="4.1">
    <config id="General">
        <prop id="smpi/os" value="0:2.9654918472829634e-07:9.636195922971908e-11;8133:0.0:0.0;15831:0.0:0.0;33956:0.0:0.0;63305:0.0:0.0"/>
        <prop id="smpi/or" value="0:1.3754300044375224e-06:8.456019002037343e-11;8133:0.0:0.0;15831:0.0:0.0;33956:0.0:0.0;63305:0.0:0.0"/>
        <prop id="smpi/ois" value="0:6.93122952108322e-07:7.051204536228499e-11;8133:3.601990790153314e-07:1.2449128895712003e-10;15831:2.977016881030904e-06:4.130261264964205e-11;33956:3.1334661540668004e-06:3.293458765282178e-11;63305:6.939993663604218e-07:0.0"/>
        <prop id="smpi/bw-factor" value="0:0.4709912642540426;8133:0.6770561715947873;15831:0.7091971477925826;33956:0.7033449540683548;63305:0.9867951082730284"/>
        <prop id="smpi/lat-factor" value="0:1.1347371681807956;8133:29.84698135078078;15831:30.274775156187765;33956:64.63917831305292;633

In [7]:
for mode in traces['mode'].unique():
    for kind in trace_reality['kind'].unique():
        df = traces[(traces['mode'] == mode) & (traces['kind'] == kind)]
        print(mode, kind, df.start.min(), df.end.max())

reality BLAS 173.488 2600.290212
reality MPI 2.8999999999999997e-05 2843.532021
simgrid BLAS 0.09401 2003.0603449999999
simgrid MPI 0.0 2003.070829


## Comparing dgemm traces

In [8]:
dgemm = traces[traces['function'] == 'dgemm'].sort_values(by=['start'])
dgemm.groupby(['rank', 'mode']).count()[['start']]

start
rank mode           
0    reality   86524
     simgrid  126894
1    reality   84853
     simgrid  130919
2    reality  180885
     simgrid  217450
3    reality  154404
     simgrid  145388
4    reality   80253
     simgrid  126474
5    reality   76905
     simgrid  144635
6    reality  199021
     simgrid  266144
7    reality  135595
     simgrid  162509

In [9]:
dgemm[(dgemm['mode'] == 'reality') & (dgemm['rank'] == 0)].head(n=20)[['m', 'n', 'k', 'start', 'end']]

,m,n,k,start,end
1,125054.0,2.0,2.0,173.488,173.488296
5,125052.0,4.0,4.0,173.489,173.489177
3,125052.0,0.0,2.0,173.489,173.489000
13,125048.0,8.0,8.0,173.490,173.490290
7,125050.0,2.0,2.0,173.490,173.490141
9,125048.0,0.0,2.0,173.490,173.490000
11,125048.0,0.0,4.0,173.490,173.490000
15,125046.0,2.0,2.0,173.491,173.491139
17,125044.0,0.0,2.0,173.492,173.492000
19,125044.0,4.0,4.0,173.492,173.492188


In [10]:
dgemm[(dgemm['mode'] == 'simgrid') & (dgemm['rank'] == 0)].head(n=20)[['m', 'n', 'k', 'start', 'end']]

,m,n,k,start,end
1,125054.0,2.0,2.0,0.094065,0.094411
3,125052.0,0.0,2.0,0.094457,0.094802
5,125052.0,4.0,4.0,0.094839,0.095188
7,125050.0,2.0,2.0,0.095235,0.095581
9,125048.0,0.0,2.0,0.095625,0.095970
11,125048.0,0.0,4.0,0.096006,0.096352
13,125048.0,8.0,8.0,0.096387,0.096751
15,125046.0,2.0,2.0,0.096797,0.097143
17,125044.0,0.0,2.0,0.097187,0.097532
19,125044.0,4.0,4.0,0.097568,0.097918


In [11]:
df = dgemm[(dgemm['mode'] == 'reality') & (dgemm['rank'] == 0)][['m', 'n', 'k']]
df.head(n=1000).to_csv('/tmp/reality.csv', index=False)

In [12]:
df = dgemm[(dgemm['mode'] == 'simgrid') & (dgemm['rank'] == 0)][['m', 'n', 'k']]
df.head(n=1000).to_csv('/tmp/simgrid.csv', index=False)

In [13]:
!git diff --no-index -- /tmp/reality.csv /tmp/simgrid.csv

diff --git a/tmp/reality.csv b/tmp/simgrid.csv
index 61a679f..3022942 100644
--- a/tmp/reality.csv
+++ b/tmp/simgrid.csv
@@ -1,48 +1,48 @@
 m,n,k
 125054.0,2.0,2.0
-125052.0,4.0,4.0
 125052.0,0.0,2.0
-125048.0,8.0,8.0
+125052.0,4.0,4.0
 125050.0,2.0,2.0
 125048.0,0.0,2.0
 125048.0,0.0,4.0
+125048.0,8.0,8.0
 125046.0,2.0,2.0
 125044.0,0.0,2.0
 125044.0,4.0,4.0
 125042.0,2.0,2.0
-125040.0,0.0,8.0
-125040.0,0.0,4.0
 125040.0,0.0,2.0
+125040.0,0.0,4.0
+125040.0,0.0,8.0
 125040.0,16.0,16.0
 125038.0,2.0,2.0
 125036.0,0.0,2.0
 125036.0,4.0,4.0
+125034.0,2.0,2.0
 125032.0,0.0,2.0
 125032.0,0.0,4.0
-125034.0,2.0,2.0
 125032.0,8.0,8.0
 125030.0,2.0,2.0
 125028.0,0.0,2.0
 125028.0,4.0,4.0
 125026.0,2.0,2.0
 125024.0,0.0,2.0
-125024.0,32.0,32.0
-125024.0,0.0,16.0
-125024.0,0.0,8.0
 125024.0,0.0,4.0
+125024.0,0.0,8.0
+125024.0,0.0,16.0
+125024.0,32.0,32.0
 125022.0,2.0,2.0
-125020.0,4.0,4.0
 125020.0,0.0,2.0
+125020.0,4.0,4.0
+125018.0,2.0,2.0
 125016.0,0.0,2.0
 125016.0,0.0,4.0
 125016.0,8.0,8.0


In [14]:
dgemm[(dgemm['rank'] == 0) & (dgemm['m'] == 123904) & (dgemm['n'] == 128) & (dgemm['k'] == 128)].groupby('mode').count()[['start']]

,start
mode,
reality,83
simgrid,107


In [15]:
dgemm[(dgemm['rank'] == 0) & (dgemm['m'] == 124160) & (dgemm['n'] == 128) & (dgemm['k'] == 128)].groupby('mode').count()[['start']]

,start
mode,
reality,77
simgrid,105


At some point in the trace diff, we see this:

In [16]:
reality_sum = 124416.0*50688.0*128.0
simgrid_sum = 124416.0*48768.0*128.0 + 124416*128*128*15
print(reality_sum)
print(simgrid_sum)

807218970624.0
807218970624.0


This is how we get a different number of event count but a same sum of size products.

## Comparing MPI_Recv traces

In [17]:
recv = traces[traces['function'] == 'MPI_Recv'].sort_values(by=['start'])
recv.groupby(['rank', 'mode']).count()[['start']]

start
rank mode          
0    reality   1984
     simgrid   1477
1    reality   2459
     simgrid   1474
2    reality   1980
     simgrid   1478
3    reality   2458
     simgrid   1474
4    reality   2470
     simgrid   2457
5    reality   1967
     simgrid   2939
6    reality   2463
     simgrid   2453
7    reality   1968
     simgrid   2937

In [18]:
recv[(recv['mode'] == 'reality') & (recv['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
3218782,4,0.005785,0.006441
3218783,4,0.006441,0.007069
3218784,4,0.007069,0.007298
3218792,8,0.007761,0.007764
3218793,8,0.007764,0.007777
3218794,8,0.007777,0.007997
3218822,12,0.010547,0.010549
3218823,12,0.010549,0.010551
3218824,12,0.010551,0.010556
3219507,128058376,174.106220,174.160124


In [19]:
recv[(recv['mode'] == 'simgrid') & (recv['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
4197212,4,0.000063,0.000064
4197214,4,0.000065,0.000067
4197215,4,0.000067,0.000068
4197221,8,0.000238,0.000242
4197222,8,0.000242,0.000245
4197223,8,0.000245,0.000249
4197241,12,0.093481,0.093482
4197242,12,0.093482,0.093531
4197243,12,0.093531,0.093533
4197982,128058376,0.208223,0.218811


In [20]:
df = recv[(recv['mode'] == 'reality') & (recv['rank'] == 0)][['msg_size']]
df.to_csv('/tmp/reality.csv', index=False)

In [21]:
df = recv[(recv['mode'] == 'simgrid') & (recv['rank'] == 0)][['msg_size']]
df.to_csv('/tmp/simgrid.csv', index=False)

In [22]:
!git diff --no-index -- /tmp/reality.csv /tmp/simgrid.csv

diff --git a/tmp/reality.csv b/tmp/simgrid.csv
index d74d042..672c3e1 100644
--- a/tmp/reality.csv
+++ b/tmp/simgrid.csv
@@ -1473,513 +1473,6 @@ msg_size
 263176
 263176
 2184
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2048
-2

## Comparing MPI_Send traces

In [23]:
send = traces[traces['function'] == 'MPI_Send'].sort_values(by=['start'])
send.groupby(['rank', 'mode']).count()[['start']]

start
rank mode          
0    reality  67513
     simgrid  66521
1    reality  66890
     simgrid  66392
2    reality  67365
     simgrid  66378
3    reality  66866
     simgrid  66371
4    reality  67020
     simgrid  65541
5    reality  67378
     simgrid  64927
6    reality  66876
     simgrid  65403
7    reality  67353
     simgrid  64908

In [24]:
send[(send['mode'] == 'reality') & (send['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
3218787,4,0.007300,0.007431
3218788,4,0.007431,0.007508
3218789,4,0.007508,0.007584
3218797,8,0.007997,0.007998
3218798,8,0.007998,0.007998
3218799,8,0.007998,0.007999
3218803,8,0.007999,0.008000
3218802,8,0.007999,0.007999
3218807,60,0.008000,0.008001
3218804,8,0.008000,0.008000


In [25]:
send[(send['mode'] == 'simgrid') & (send['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
4197217,4,0.000071,0.000071
4197218,4,0.000071,0.000071
4197219,4,0.000071,0.000072
4197226,8,0.000250,0.000251
4197225,8,0.000250,0.000250
4197227,8,0.000251,0.000251
4197228,8,0.000251,0.000251
4197229,8,0.000251,0.000251
4197230,8,0.000251,0.000252
4197233,60,0.000252,0.000253


In [26]:
df = send[(send['mode'] == 'reality') & (send['rank'] == 0)][['msg_size']]
df.head(n=1000).to_csv('/tmp/reality.csv', index=False)

In [27]:
df = send[(send['mode'] == 'simgrid') & (send['rank'] == 0)][['msg_size']]
df.head(n=1000).to_csv('/tmp/simgrid.csv', index=False)

In [28]:
!git diff --no-index -- /tmp/reality.csv /tmp/simgrid.csv

diff --git a/tmp/reality.csv b/tmp/simgrid.csv
index ede87c1..b2b58b4 100644
--- a/tmp/reality.csv
+++ b/tmp/simgrid.csv
@@ -7,10 +7,10 @@ msg_size
 8
 8
 8
-60
 8
 60
 60
+60
 44
 44
 44
@@ -151,10 +151,10 @@ msg_size
 128058376
 128058376
 63963136
-32981520
+499720
 127927304
 63963136
-70176
+1032
 2080
 2080
 2080
@@ -285,12 +285,12 @@ msg_size
 2080
 127796232
 127796232
-33911328
+498696
 63832064
 127665160
-35906112
+498696
 63832064
-62952
+1032
 2080
 2080
 2080
@@ -421,12 +421,12 @@ msg_size
 2080
 127534088
 127534088
-30357992
+497672
 63700992
 127403016
-29860320
+497672
 63700992
-66048
+1032
 2080
 2080
 2080
@@ -557,12 +557,12 @@ msg_size
 2080
 127271944
 127271944
-31785472
+496648
 63569920
 127140872
-28308936
+496648
 63569920
-59856
+1032
 2080
 2080
 2080
@@ -693,12 +693,12 @@ msg_size
 2080
 127009800
 127009800
-28746192
+495624
 63438848
 126878728
-34693680
+495624
 63438848
-69144
+1032
 2080
 2080
 2080
@@ -829,12 +829,12 @@ msg_size
 2080
 126747656
 12

## Comparing MPI_Irecv traces

In [29]:
irecv = traces[traces['function'] == 'MPI_Irecv'].sort_values(by=['start'])
irecv.groupby(['rank', 'mode']).count()[['start']]

start
rank mode          
0    reality  65031
     simgrid  64055
1    reality  64922
     simgrid  63456
2    reality  64901
     simgrid  63926
3    reality  64902
     simgrid  63439
4    reality  65031
     simgrid  64055
5    reality  64922
     simgrid  63456
6    reality  64901
     simgrid  63926
7    reality  64902
     simgrid  63439

In [30]:
irecv[(irecv['mode'] == 'reality') & (irecv['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
3218836,1056,173.489695,173.489699
3218839,1056,173.498091,173.498097
3218844,1056,173.498696,173.498698
3218847,1056,173.499140,173.499142
3218854,1056,173.499686,173.499687
3218857,1056,173.500057,173.500059
3218862,1056,173.500450,173.500451
3218865,1056,173.500764,173.500765
3218874,1056,173.501359,173.501361
3218877,1056,173.501707,173.501708


In [31]:
irecv[(irecv['mode'] == 'simgrid') & (irecv['rank'] == 0)].head(n=20)[['msg_size', 'start', 'end']]

,msg_size,start,end
4197256,1056,0.094353,0.094353
4197261,1056,0.094362,0.094362
4197268,1056,0.094779,0.094779
4197272,1056,0.094783,0.094783
4197279,1056,0.095556,0.095556
4197283,1056,0.095562,0.095562
4197288,1056,0.095948,0.095948
4197292,1056,0.095951,0.095951
4197301,1056,0.097118,0.097118
4197305,1056,0.097124,0.097124


In [32]:
df = irecv[(irecv['mode'] == 'reality') & (irecv['rank'] == 0)][['msg_size']]
df.head(n=2000).to_csv('/tmp/reality.csv', index=False)

In [33]:
df = irecv[(irecv['mode'] == 'simgrid') & (irecv['rank'] == 0)][['msg_size']]
df.head(n=2000).to_csv('/tmp/simgrid.csv', index=False)

In [34]:
!git diff --no-index -- /tmp/reality.csv /tmp/simgrid.csv

diff --git a/tmp/reality.csv b/tmp/simgrid.csv
index 95857cc..c6f7417 100644
--- a/tmp/reality.csv
+++ b/tmp/simgrid.csv
@@ -127,9 +127,7 @@ msg_size
 1056
 1056
 1056
-29983200
 63963136
-36479560
 131072
 1056
 1056
@@ -260,9 +258,7 @@ msg_size
 1056
 1056
 63832064
-29423064
 63832064
-33911328
 131072
 1056
 1056
@@ -393,9 +389,7 @@ msg_size
 1056
 1056
 63700992
-29860320
 63700992
-28864976
 131072
 1056
 1056
@@ -526,9 +520,7 @@ msg_size
 1056
 1056
 63569920
-32282120
 63569920
-29302232
 131072
 1056
 1056
@@ -659,9 +651,7 @@ msg_size
 1056
 1056
 63438848
-27754944
 63438848
-31224312
 131072
 1056
 1056
@@ -792,9 +782,7 @@ msg_size
 1056
 1056
 63307776
-30665200
 63307776
-34622000
 131072
 1056
 1056
@@ -925,9 +913,7 @@ msg_size
 1056
 1056
 63176704
-32576016
 63176704
-31588864
 131072
 1056
 1056
@@ -1058,9 +1044,7 @@ msg_size
 1056
 1056
 63045632
-31030776
 63045632
-30045672
 131072
 1056
 1056
@@ -1191,9 +1175,7 @@ msg_size
 1056
 1056
 62914560
-34898488
 62914560
